In [22]:
# Map Castilla y León cultural agenda geolocalized
import os
import folium  # Maps library
import requests # Library to make url requests
import time
import webbrowser
import bs4 # Web Scrapping
import random
import base64
%load_ext folium_magic

The folium_magic extension is already loaded. To reload it, use:
  %reload_ext folium_magic


In [23]:
# Web Scrapping

#cyl = "https://datosabiertos.jcyl.es/web/es/datos-abiertos-castilla-leon.html"
cyl = "https://datosabiertos.jcyl.es/web/jcyl/RISP/es/PlantillaBuscadorContenidos/1284162104333/Dataset/1284197734502/_?p0=false&p2=Dataset&tituloc=true&q=empleos&aceptar=Buscar"
def scrap(url):
    r  = requests.get(url)

    data = r.text

    soup = bs4.BeautifulSoup(data, "html.parser")
    
    return soup
soup = scrap(cyl)

# Locating website Ofertas de Empleo dataset
empleo = [x.get('href') for x in soup.findAll('a') if 'ofertas' in x.get('href')][0]
# Locating JSON file
soup = scrap(empleo)
cyl_json = [link.get('href') for link in soup.findAll('a') if "json" in link.get('href')][0]
print(cyl_json)


https://datosabiertos.jcyl.es/web/jcyl/risp/es/empleo/ofertas-empleo/1284354353012.json


In [24]:
"""
while True:
"""   
# Requesting the file
r = requests.get(cyl_json)
    
# Having the file as a JSON object
r = r.json()

#date = r['document']['date']
empleos, geolocs = r['document']['list'], []
for i in empleos:
    if 'LocalidadPadre' in i['element']['attribute'][9]:
        geolocs.append(i)
        #print(i['element']['attribute'][9]['LocalidadPadre'])
#print(geolocs[0]['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1])
no_geoloc = [i for i in empleos if not i in geolocs]

In [39]:
# Creating a map
centro = geolocs[0]['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1]
centro_posicion = [float(centro[0]),float(centro[1])]
folium_map = folium.Map(center=centro_posicion , zoom_start = 15000)
markers = []
# Creating a circle on the map
for i in geolocs:
    titulo = i['element']['attribute'][1]['text']
    fecha = i['element']['attribute'][3]['date']
    content = '<a href="{}">Enlace</a>'.format((i['element']['attribute'][-1]['valor']))
    latlong = i['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1]
    link = content.format(link = '{}'.format(content), text='{}'.format('Enlace'))
    print(link)
    if latlong!= []:
        posicion = [float(latlong[0]),float(latlong[1])]
    folium.Circle(
        radius=100,
        location= posicion,
        popup= link, 
        tooltip= titulo+'\n\n'+ fecha,
        color= '#'+str(random.randint(0,999999)),
        fill=True,
        ).add_to(folium_map)
    markers.append({'lat': posicion[0],'lng': posicion[1], "popup": titulo+'\n\n'+ fecha+'\n\n'+link})
print(str(len(markers))+ " trabajos en Castilla & León")   

<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139476/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139376/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139308/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139262/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139219/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139175/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/1284216458054/OfertasEmpleo/1284946139127/Empleo">Enlace</a>
<a href="https://empleo.jcyl.es/web/jcyl/Empleo/es/Plantilla100Detalle/12842

In [37]:
%folium_map -M markers

In [18]:
# Saving the map to a HTML file to be read by a web browser
pagina = folium_map.save('cyl_empleos.html')

In [19]:
# Run html in webbrowser
webbrowser.open('file://'+ os.path.realpath('cyl_empleos.html'), new = -1)

False